In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
import random
from utils import *
from collections import defaultdict
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

### Hyperparameter of implementation

In [3]:
anchor_probability_numerator = 20
notifications = 2
epsilon = 0.2
deletion_prob_at_each_step = 0.2
num_samples_for_agreement_and_heavyness_calculation = 2
num_samples_for_connect_procedure = 2
num_experiments = 1

In [4]:
# Set Seaborn color palette for color-blind friendly colors
sns.set_palette("colorblind")

#### The list datasets contains the relative location of each dataset. Each line of the dataset contains two comma separated numbers which correspond to an edge between two nodes, e.g., "84424, 276"  


In [5]:
# datasets contains the relative location of the datasets. Each line of the dataset contains 
datasets =  ["datasets/musae-facebook.csv",  "datasets/Email-Enron.csv", "datasets/Cit-HepTh.csv", "datasets/CA-AstroPh.csv"]

In [7]:
for csv_file_path in datasets:
    # Create lists to store objective values for each dataset
    graph_adjacency_lists = create_graph_from_csv(csv_file_path)
    # Generate a random node arrival order
    random_node_order = generate_random_node_order(graph_adjacency_lists)

    # Initialization for the random variables of dynamic pivot
    # we first contract a random permulation pi: node --> order in the random permulation
    pi = {element: index for index, element in enumerate(generate_random_node_order(graph_adjacency_lists))}
    eta = {key: key for key in pi.keys()}
    pivot_clustering = {key: key for key in pi.keys()}

    # Initializaiton for the random variables of agreement
    sparse_graph = {}
    nodes_present_at_the_moment = OptList()
    Phi = set()
    Phi_nodes = {}
    I_nodes = defaultdict(set)
    B_nodes = {node:OptList() for node in graph_adjacency_lists}


    # Initialize an empty dictionary to store the current graph
    current_graph = {}
    # p is the probabilty of a random deletion at each time step
    random_node_iterator = iter(random_node_order)
    i = -1
    only_deletion = False
    total_iterations = 2* len(graph_adjacency_lists)
    #### Here starts the copy paste

    while True:
        i +=1
        if (i > 100) and (not current_graph):
            print("Oh, now the graph is empty")
            break
        deletion = False
        try:
            # node addition
            node = next(random_node_iterator)
        except StopIteration:
            # StopIteration is raised when the iterator is exhausted
            clustering = connected_components(sparse_graph)
            all_singletons = {element: index for index, element in enumerate(sparse_graph.keys())}
            pivot_clustering_this_step = {u: pivot_clustering[u] for u in sparse_graph.keys()}
            classical_pivot_clustering_sol = classical_pivot(graph_adjacency_lists)
            
            # Calculate correlation values and store them
            corr_clustering = correlation_clustering_value(current_graph, clustering)
            corr_all_singletons = correlation_clustering_value(current_graph, all_singletons)
            corr_pivot_clustering = correlation_clustering_value(current_graph, pivot_clustering_this_step)
            classical_pivot_clustering = correlation_clustering_value(current_graph, classical_pivot_clustering_sol)
            

            correlation_values_clustering = corr_clustering/corr_all_singletons
            correlation_values_all_singletons = corr_all_singletons/corr_all_singletons
            correlation_values_pivot_clustering = corr_pivot_clustering/corr_all_singletons
            correlation_values_classical_pivot_clustering = classical_pivot_clustering/corr_all_singletons
            break        
        current_graph[node] = OptList()
        nodes_present_at_the_moment.append(node)
        # Add edges to previously arrived nodes
        for neighbor in graph_adjacency_lists[node]:
            if neighbor in nodes_present_at_the_moment:
                # I have to update the pivot clustering
                # this is the case where node may become the new pivot of neighbor
                if pi[eta[neighbor]] > pi[node]:
                    if eta[neighbor] == neighbor:
                        for w in current_graph[neighbor]:
                            if w == neighbor:
                                continue
                            pivot_clustering[w] = w if eta[w]==neighbor else pivot_clustering[w]
                    eta[neighbor] = node
                    pivot_clustering[neighbor] = node if eta[node]==node else neighbor

                if pi[eta[node]] > pi[neighbor]:
                    if eta[node] == node:
                        for w in current_graph[node]:
                            if w == node:
                                continue
                            pivot_clustering[w] = w if eta[w]==node else pivot_clustering[w]
                    eta[node] = neighbor
                    pivot_clustering[node] = neighbor if eta[neighbor]==neighbor else node
                current_graph[neighbor].append(node)
                current_graph[node].append(neighbor)
        # Notify
        # ---------send Type 0 notifications-------------------
        notified_so_far = set()
        notified_so_far.add(node)
        got_type_0_notification = current_graph[node].getRandom(notifications)
        if got_type_0_notification:
            got_type_0_notification.discard(node)
            I_nodes[node].update(got_type_0_notification)
            for v in got_type_0_notification:
                B_nodes[v].append(node)

        # ---------send Type 1 notifications-------------------
        got_type_1_notification = set()
        for v in got_type_0_notification:
            if v not in current_graph:
                continue
            # Avoid that the notification graph becomes too dense
            for u in I_nodes[v]:
                B_nodes[u].remove(v)
            # take another sample 1
            br = current_graph[v].getRandom(notifications)
            I_nodes[v].update(current_graph[v].getRandom(notifications))
            I_nodes[v].discard(v)
            I_nodes[v].discard(node)
            for u in I_nodes[v]:
                B_nodes[u].append(v)
                if u not in notified_so_far:
                    got_type_1_notification.add(u)
                    notified_so_far.add(u)
        # ---------send Type 2 notifications-------------------
        got_type_2_notification = set()
        for v in got_type_1_notification:
            if v not in current_graph:
                continue
            for u in I_nodes[v]:
                B_nodes[u].remove(v)
            # take another sample 2
            br = current_graph[v].getRandom(notifications)
            I_nodes[v].update(current_graph[v].getRandom(notifications))
            I_nodes[v].discard(v)
            I_nodes[v].discard(node)
            for u in I_nodes[v]:
                B_nodes[u].append(v)
                if (u not in notified_so_far) or (u not in got_type_2_notification) :
                    got_type_2_notification.add(u)
        # ---------receive Type 2 notifications-------------------
        for v in got_type_2_notification:
            if v not in current_graph:
                continue
            # Update the sample
            for u in I_nodes[v]:
                B_nodes[u].remove(v)
            I_nodes[v] = current_graph[v].getRandom(notifications)
            I_nodes[v].discard(v)
            I_nodes[v].discard(node)
            for u in I_nodes[v]:
                B_nodes[u].append(v)
        # First we check if it is a deletion or addition and initialize the respective variables
        sparse_graph[node] = OptList()
        Phi_nodes[node] = set()
        # The interesting event set corresponds to the notified_so_far
        for u in notified_so_far:
            # If u is in Phi we first delete all its edges to nodes not in Phi
            if u in Phi:
                neighbors_of_u = [v for v in sparse_graph[u]]
                for v in neighbors_of_u:
                    if (u == v) or (v in Phi):
                        continue
                    sparse_graph[u].remove(v)
                    sparse_graph[v].remove(u)
                    Phi_nodes[v].discard(u)
            Phi.discard(u)
            # Anchor procedure
            anchor_prob = anchor_probability_numerator / len(current_graph[u])
            if (random.random() < anchor_prob) and ProbAHeaviness(u, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                Phi.add(u)
                for v in current_graph[u]:
                    if ProbAgreement(u, v, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                        sparse_graph[u].append(v)
                        sparse_graph[v].append(u)
                        if u != v:
                            Phi_nodes[v].add(u)
            # Clean procedure
            connected_nodes_in_anchor_set = [w for w in Phi_nodes[u]]
            for w in connected_nodes_in_anchor_set:
                if w not in nodes_present_at_the_moment:
                    Phi_nodes[u].discard(w)
                    continue
                if w == u:
                    continue
                if ProbAgreement(u, w, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation) and ProbAHeaviness(w, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                    continue
                else:
                    sparse_graph[u].remove(w)
                    sparse_graph[w].remove(u)
                    Phi_nodes[u].discard(w)
            # Connect procedure
            J_u = current_graph[u].getRandom(num_samples_for_connect_procedure)
            for w in J_u:
                phi_nodes_w = [r for r in Phi_nodes[w]]
                for r in phi_nodes_w:
                    if r not in nodes_present_at_the_moment:
                        Phi_nodes[w].discard(r)
                        continue
                    if r not in current_graph[u]:
                        continue
                    if ProbAgreement(u, r, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation) and ProbAHeaviness(r, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                        sparse_graph[u].append(r)
                        sparse_graph[r].append(u)
                        Phi_nodes[u].add(r)

        # Plot the results
        
    
    print(f"agreement = {correlation_values_clustering} vs dynamic pivot = {correlation_values_pivot_clustering} vs singleton = {correlation_values_all_singletons} vs classical pivot = {correlation_values_classical_pivot_clustering}")


agreement = 0.9691669154622036 vs dynamic pivot = 1.120703886478987 vs singleton = 1.0 vs classical pivot = 1.0929909906745578
agreement = 0.9536857222122493 vs dynamic pivot = 1.01882163508875 vs singleton = 1.0 vs classical pivot = 1.0721097094614074
agreement = 0.9977291049267069 vs dynamic pivot = 1.1944283589376752 vs singleton = 1.0 vs classical pivot = 1.2438998081093664
agreement = 0.9322292350416561 vs dynamic pivot = 1.0044837162332745 vs singleton = 1.0 vs classical pivot = 1.247760666498359
